In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.config.experimental import list_physical_devices

# Checking GPU availability
print("Num GPUs Available: ", len(list_physical_devices('GPU')))

# Loading pre-processed data from files
X = np.load('/content/combined_features.npy')
y = np.load('/content/combined_labels.npy')

# Converting class labels to numerical values
emotion_to_int = {emotion: idx for idx, emotion in enumerate(set(y))}
y_int = np.array([emotion_to_int[emotion] for emotion in y])
y_one_hot = np.zeros((len(y), len(emotion_to_int)))
for i, emotion in enumerate(y_int):
    y_one_hot[i, emotion] = 1

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Checking data shapes
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

# Defining the enhanced model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Using early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluating the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy:.2%}')


Num GPUs Available:  1
Shape of X_train: (23091, 13, 174)
Shape of y_train: (23091, 5)
Shape of X_test: (5773, 13, 174)
Shape of y_test: (5773, 5)
Epoch 1/100
578/578 [==============================] - 14s 11ms/step - loss: 1.2505 - accuracy: 0.4121 - val_loss: 1.1452 - val_accuracy: 0.4780
Epoch 2/100
578/578 [==============================] - 6s 11ms/step - loss: 1.1445 - accuracy: 0.4794 - val_loss: 1.0853 - val_accuracy: 0.5086
Epoch 3/100
578/578 [==============================] - 5s 9ms/step - loss: 1.0970 - accuracy: 0.5039 - val_loss: 1.0543 - val_accuracy: 0.5360
Epoch 4/100
578/578 [==============================] - 6s 10ms/step - loss: 1.0646 - accuracy: 0.5267 - val_loss: 1.0143 - val_accuracy: 0.5562
Epoch 5/100
578/578 [==============================] - 5s 9ms/step - loss: 1.0216 - accuracy: 0.5554 - val_loss: 0.9886 - val_accuracy: 0.5819
Epoch 6/100
578/578 [==============================] - 5s 9ms/step - loss: 0.9895 - accuracy: 0.5727 - val_loss: 0.9337 - val_accuracy

In [ ]:
# Saving the trained model (can be used after calling model.fit)
model.save("/content/emotion_model.h5")

print("Model saved as '/content/emotion_model.h5'")

Model saved as '/content/emotion_model.h5'


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Saving model weights
model.save_weights("/content/emotion_model_weights.h5")

# For loading weights
# model.load_weights("/content/emotion_model_weights.h5")

In [ ]:
from google.colab import files

# Exporting the model file
files.download('/content/emotion_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Exporting the model file
files.download('/content/emotion_model_weights.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Saving the model in the new Keras format
model.save("/content/emotion_model.keras")

In [ ]:
# Exporting the model file
files.download('/content/emotion_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import librosa
from tensorflow.keras.models import load_model
import os

# Loading the model
model_path = 'emotion_model.keras'
model = load_model(model_path)

# Dictionary for decoding emotions
emotion_to_int = {0: 'angry', 1: 'happy', 2: 'neutral', 3: 'sad', 4: 'surprised'}
# Reverse: {v: k for k, v in emotion_to_int.items()}
int_to_emotion = {v: k for k, v in emotion_to_int.items()}

# Function for preprocessing audio files
def preprocess_audio(file_path, max_pad_len=174):
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    audio_data = audio_data / np.max(np.abs(audio_data))
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
    if mfccs.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_pad_len]
    return mfccs

# Path to audio files
audio_files = ['Surprised.wav', 'Sad.wav']
audio_dir = './'

# Preprocessing and predictions
for file_name in audio_files:
    file_path = os.path.join(audio_dir, file_name)
    mfcc_features = preprocess_audio(file_path)
    mfcc_features = np.expand_dims(mfcc_features, axis=0)  # Adding a batch dimension
    prediction = model.predict(mfcc_features)
    predicted_emotion = emotion_to_int[np.argmax(prediction)]
    print(f"{file_name}: {predicted_emotion}")

# Additionally, if probabilities are needed
    probabilities = prediction[0]
    emotion_probabilities = {emotion_to_int[i]: prob for i, prob in enumerate(probabilities)}
    print(f"Probabilities for {file_name}: {emotion_probabilities}\n")

1/1 [==============================] - 1s 897ms/step
Surprised.wav: surprised
Probabilities for Surprised.wav: {'angry': 6.5860775e-05, 'happy': 0.07020777, 'neutral': 0.011884412, 'sad': 0.1126476, 'surprised': 0.8051943}

1/1 [==============================] - 0s 25ms/step
Sad.wav: sad
Probabilities for Sad.wav: {'angry': 2.349024e-06, 'happy': 0.000814713, 'neutral': 0.0006074288, 'sad': 0.9939521, 'surprised': 0.0046234373}

